In [1]:
import nltk 
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize, sent_tokenize 
import numpy as np
import math
from rouge import Rouge 
import string
import re
from nltk.stem import PorterStemmer 
from rouge import Rouge 
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

In [34]:
def metinoku(i):
    no=str(i)
    a="ent/ham/"
    b=".txt"
    return a+no+b
def özetal(i):
    no=str(i)
    a="ent/özet/"
    b=".txt"
    ada=a+no+b
    txt=open(ada,"r")
    f=txt.read()
    f.endswith("\n")
    return f

In [35]:
freq=dict()
leng=387

In [41]:
def çıkar(i):
    c=i
    ada=metinoku(i)
    txt=open(ada,"r")
    f=txt.read()
    tokenize_sent = sent_tokenize(f) 
    for i in range(0,len(tokenize_sent)):
        tokenize_sent[i]=re.sub(r'[^\w\s]','',tokenize_sent[i])
        
    adj_matrix= np.zeros((len(tokenize_sent),len(tokenize_sent))) #Cümle sayısına göre 
    sent_rank = np.random.rand(len(tokenize_sent)) #Cümle değerleri için tuttuğum değişken

#Stopword'lerin kaldırılması
    stop = stopwords.words('english')
    process_sent=list()
    
    for i in range(0,len(tokenize_sent)):
        if i not in stop:
            process_sent.append(tokenize_sent[i].split())
    freq=dict()
    copy_sent=process_sent
    ps= PorterStemmer()
    for sentence_in_running,i in enumerate(process_sent):
        for word_in_running,j in enumerate(i):
            copy_sent[sentence_in_running][word_in_running]=ps.stem(j)
            
    for sentence in copy_sent:
        a=len(sentence)
        for word in sentence:
            if word in freq:
                freq[word]=freq[word]+1
                freq[word]=freq[word]/a
            else :
                freq[word]=1
                freq[word]/a
    idfdata=dict()
    def find_idf(a):
        for sentence in copy_sent:
            for word in sentence:
                if a==word:
                    idfdata[a]=idfdata[a]+1
            pass

    for sentence in copy_sent:
        for word in sentence:
            idfdata[word]=0
            find_idf(word)
            
    a=len(sentence)        
    idfdeger=dict()
    for key in idfdata:
        idfdeger[key]=math.log(a/idfdata[key])
    
    for sentence in copy_sent:
        for word in sentence:
            freq[word]=(freq[word]*idfdeger[word])        
    def cosinus(a,b):
        value=[a,b]
        count_vectorizer = CountVectorizer()
        sparse_matrix = count_vectorizer.fit_transform(value)
        doc_term_matrix = sparse_matrix.todense()
        df = pd.DataFrame(doc_term_matrix,
                  columns=count_vectorizer.get_feature_names(), 
                  index= [value])
    #print(df.head())
        cos = cosine_similarity(df, df)
        cos2=cos[0][1]
    #print(cosDf.head())
        return cos2
    
    for i in range(0,len(tokenize_sent)):
        for j in range(i+1,len(tokenize_sent)):
            adj_matrix[i,j] = adj_matrix[j,i] = cosinus(tokenize_sent[i],tokenize_sent[j])
    cümleler={}
    for i in range(len(copy_sent)):
        cümleler[i]=0
    
    for i in range(0, len(copy_sent)):
        for j in range(0,len(copy_sent[i])):
            cümleler[i]=cümleler[i]+freq[copy_sent[i][j]]
    
    def calculate_weight(sent_rank,adjacency,shp,cümleler):
        sent_rank = np.array(sent_rank)
        for node1 in range(0,shp[0]):
            sent_rank[node1] = 0
            for node2 in range(0,shp[0]):
                sent_rank[node1] = (adj_matrix[node1,node2]*cümleler[node1])**2+sent_rank[node1]
            
        return sent_rank
    sent_rank = calculate_weight(sent_rank,adj_matrix,adj_matrix.shape,cümleler)
    sentences_rank=(sent_rank).argsort()
    
    summarization_sorted=np.sort(sentences_rank[1:7])
    
    system_sum=""
    for i in range(0,len(summarization_sorted)):
        system_sum=system_sum+tokenize_sent[summarization_sorted[i]]+"."

    hyp=system_sum
    
    ref=özetal(c)
    rouge=Rouge()
    scores = rouge.get_scores(hyp, ref,avg=True)
    return scores

In [42]:
rouge_1_f=0
rouge_1_p=0
rouge_1_r=0
rouge_2_f=0
rouge_2_p=0
rouge_2_r=0
rouge_l_f=0
rouge_l_p=0
rouge_l_r=0
for x in range(1,leng):
    scores=çıkar(x)
    rouge_1_f=scores['rouge-1']['f']+rouge_1_f
    rouge_1_p=scores['rouge-1']['p']+rouge_1_p
    rouge_1_r=scores['rouge-1']['r']+rouge_1_r
    rouge_2_f=scores['rouge-2']['f']+rouge_2_f
    rouge_2_p=scores['rouge-2']['p']+rouge_2_p
    rouge_2_r=scores['rouge-2']['r']+rouge_2_r
    rouge_l_f=scores['rouge-l']['f']+rouge_l_f
    rouge_l_p=scores['rouge-l']['p']+rouge_l_p
    rouge_l_r=scores['rouge-l']['r']+rouge_l_r

In [43]:
rouge_1_f/leng

0.4602194811921654

In [44]:
rouge_2_f/leng

0.3183550305228785

In [45]:
rouge_l_f/leng

0.4604316568330196

In [ ]:
 idfdata=dict()
    def find_idf(a):
        for sentence in copy_sent:
            for word in sentence:
                if a==word:
                    idfdata[a]=idfdata[a]+1
            pass

    for sentence in copy_sent:
        for word in sentence:
            idfdata[word]=0
            find_idf(word)
            
    a=len(sentence)        
    idfdeger=dict()
    for key in idfdata:
        idfdeger[key]=math.log(a/idfdata[key])
    
    for sentence in copy_sent:
        for word in sentence:
            freq[word]=(freq[word]*idfdeger[word])